In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import mlflow
import mlflow.pytorch

In [10]:
# Define simple neural network
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.flatten(x, start_dim=1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_set = torchvision.datasets.MNIST(root='../data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)


In [12]:
# Function to train the model
def train(model, train_loader, criterion, optimizer, n_epochs=3):
    model.train()
    for epoch in range(n_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        
        # Log epoch loss to MLflow
        mlflow.log_metric("loss", running_loss / len(train_loader), step=epoch)
        print(f"Epoch [{epoch+1}/{n_epochs}], Loss: {running_loss/len(train_loader):.4f}")

In [ ]:
# Initialize MLflow experiment
mlflow.set_tracking_uri("http://127.0.0.1:5000")  # Ensure the URI is the same as your local server
mlflow.set_experiment("pytorch-mnist-demo")

# Start MLflow run
with mlflow.start_run() as run:
    # Log parameters
    n_epochs = 5
    lr = 0.002
    mlflow.log_param("epochs", n_epochs)
    mlflow.log_param("learning_rate", lr)

    # Define model, loss function, and optimizer
    model = SimpleNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Train the model
    train(model, train_loader, criterion, optimizer, n_epochs)

    # Log model to MLflow
    mlflow.pytorch.log_model(model, "model")

    print(f"Model logged with run_id: {run.info.run_id}")


# Load the model

In [15]:
import mlflow.pytorch

# Load the model from a specific run_id and artifact path
run_id = "1c85debc1f8c46bbad85cd769676e104"
model_uri = f"runs:/{run_id}/model"

# Load the PyTorch model
model = mlflow.pytorch.load_model(model_uri)



In [ ]:
# Use the model for inference
model.eval()  # Set model to evaluation mode

dummy_input = torch.randn(1, 28 * 28)  # Assuming the model is trained on 28x28 images flattened
output = model(dummy_input)
print(output)